In [131]:
import codecs

import numpy as np
from sklearn.linear_model import LinearRegression
import numpy
import numpy as np
import pandas as pd
import math 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

In [132]:
# Тест Чоу позволяет оценить значимость улучшения регрессионной модели после разделения исходной выборки на части. 
# Это модификация однопараметрической экспоненциальной модели с коррекцией коэффициента линейного тренда. 
# В методе Чоу происходит адаптация параметра к изменениям в дианамике ряда.

In [133]:
def F1(n,e,e1,e2,p):
    return ((e-e1-e2)*(n-2*p-2))/((e1-e2)*(p+1))

def F2(n,e,e1,e2,p):
    return ((e-e1-e2)*(n-2*p-2))/((e1+e2)*(p+1))

In [134]:
df = pd.read_csv('chowtest_df.csv', encoding='latin-1')
df

sex  age            education   job_title  salary
0    M   52               Middle  Specialist    9000
1    M   35               Middle  Specialist   10500
2    M   18  Secondary technical   Dependent    8500
3    M   48               Higher  Supervisor   18000
4    M   50               Higher  Specialist   13000
5    M   38  Secondary technical  Supervisor   17000
6    M   43               Middle      Worker   13000
7    M   24               Middle      Worker   14000
8    M   49               Middle   Dependent   15000
9    M   35               Higher  Specialist   14000
10   M   33               Higher  Supervisor   16000
11   M   30               Middle      Worker   12000
12   M   37               Higher   Dependent   11000
13   F   20  Secondary technical      Worker    9000
14   F   39               Middle  Supervisor   17000
15   F   25  Secondary technical  Specialist   12000
16   F   32               Middle      Worker   10000
17   F   23  Secondary technical   Dependent    9000
18   F   40               Higher  Supervisor   19500
19   F   23  Secondary technical   Dependent    7000
20   F   28               Middle   Dependent    8500
21   F   37               Middle      Worker   11000
22   F   22               Higher      Worker   13000
23   F   25               Higher   Dependent    7000
24   F   45               Middle  Supervisor   17000
25   F   43               Middle  Specialist   15000
26   F   20               Higher      Worker    9000
27   F   21  Secondary technical      Worker   10000
28   F   32               Higher  Specialist   15000
29   F   33               Middle  Specialist   13000

In [135]:
# Отбираем данные для рассчёта остаточной суммы квадратов всего интервала

y=np.array(df['salary'])
x1=np.array(df['age'])

In [136]:


job_title_group = []
for i in range(len(y)):
        if(df['job_title'][i]=="Supervisor"):
            job_title_group.append(4)
            
        if(df['job_title'][i]=="Specialist"):
            job_title_group.append(3)
            
        if(df['job_title'][i]=="Dependent"):
            job_title_group.append(2)  
            
        if(df['job_title'][i]=="Worker"):
            job_title_group.append(1) 
            
df['group'] = job_title_group
display(df)

sex  age            education   job_title  salary  group
0    M   52               Middle  Specialist    9000      3
1    M   35               Middle  Specialist   10500      3
2    M   18  Secondary technical   Dependent    8500      2
3    M   48               Higher  Supervisor   18000      4
4    M   50               Higher  Specialist   13000      3
5    M   38  Secondary technical  Supervisor   17000      4
6    M   43               Middle      Worker   13000      1
7    M   24               Middle      Worker   14000      1
8    M   49               Middle   Dependent   15000      2
9    M   35               Higher  Specialist   14000      3
10   M   33               Higher  Supervisor   16000      4
11   M   30               Middle      Worker   12000      1
12   M   37               Higher   Dependent   11000      2
13   F   20  Secondary technical      Worker    9000      1
14   F   39               Middle  Supervisor   17000      4
15   F   25  Secondary technical  Specialist   12000      3
16   F   32               Middle      Worker   10000      1
17   F   23  Secondary technical   Dependent    9000      2
18   F   40               Higher  Supervisor   19500      4
19   F   23  Secondary technical   Dependent    7000      2
20   F   28               Middle   Dependent    8500      2
21   F   37               Middle      Worker   11000      1
22   F   22               Higher      Worker   13000      1
23   F   25               Higher   Dependent    7000      2
24   F   45               Middle  Supervisor   17000      4
25   F   43               Middle  Specialist   15000      3
26   F   20               Higher      Worker    9000      1
27   F   21  Secondary technical      Worker   10000      1
28   F   32               Higher  Specialist   15000      3
29   F   33               Middle  Specialist   13000      3

In [137]:
salary = []
for i in range(len(df)):
    salary.append(df['salary'][i])
df['salary'] = salary
df

sex  age            education   job_title  salary  group
0    M   52               Middle  Specialist    9000      3
1    M   35               Middle  Specialist   10500      3
2    M   18  Secondary technical   Dependent    8500      2
3    M   48               Higher  Supervisor   18000      4
4    M   50               Higher  Specialist   13000      3
5    M   38  Secondary technical  Supervisor   17000      4
6    M   43               Middle      Worker   13000      1
7    M   24               Middle      Worker   14000      1
8    M   49               Middle   Dependent   15000      2
9    M   35               Higher  Specialist   14000      3
10   M   33               Higher  Supervisor   16000      4
11   M   30               Middle      Worker   12000      1
12   M   37               Higher   Dependent   11000      2
13   F   20  Secondary technical      Worker    9000      1
14   F   39               Middle  Supervisor   17000      4
15   F   25  Secondary technical  Specialist   12000      3
16   F   32               Middle      Worker   10000      1
17   F   23  Secondary technical   Dependent    9000      2
18   F   40               Higher  Supervisor   19500      4
19   F   23  Secondary technical   Dependent    7000      2
20   F   28               Middle   Dependent    8500      2
21   F   37               Middle      Worker   11000      1
22   F   22               Higher      Worker   13000      1
23   F   25               Higher   Dependent    7000      2
24   F   45               Middle  Supervisor   17000      4
25   F   43               Middle  Specialist   15000      3
26   F   20               Higher      Worker    9000      1
27   F   21  Secondary technical      Worker   10000      1
28   F   32               Higher  Specialist   15000      3
29   F   33               Middle  Specialist   13000      3

In [138]:
# Отбираем остаток данных для всего интервала

x2=np.array(df['group'])
X = df[['group', 'age']]

In [139]:
# Добавляем константу в переменные-предсказатели
X = sm.add_constant(X)

#Подбираем модель линейной регрессии для всех
model_all = sm.OLS(y, X).fit()

In [140]:
# Остаточная сумма квадратов для всего интервала

RSS_all = np.sum((model_all.fittedvalues - np.array(y).mean())**2)
print(ssr_all)

164724927.86061692


In [141]:
# Отбираем только данные для женщин

y=np.array(df['salary'])
y=y[0:15]
X=df[['group', 'age']]
X=X[0:15]

In [142]:
# Добавляем константу в переменные-предсказатели
X = sm.add_constant(X)

#Подбираем модель линейной регрессии для мужчин
model_man = sm.OLS(y, X).fit()

In [143]:
# Остаточная сумма квадратов для всего мужчин [0:15]

RSS_man = np.sum((model_man.fittedvalues - np.array(y).mean())**2)
print(ssr_man)

46246098.187017165


In [144]:
# Отбираем только данные для женщин

y=np.array(df['salary'])
y=y[15:30]
X=df[['group', 'age']]
X=X[15:30]

In [145]:
# Добавляем константу в переменные-предсказатели
X = sm.add_constant(X)

#Подбираем модель линейной регрессии для женщин
model_woman = sm.OLS(y, X).fit()

In [146]:
# Остаточная сумма квадратов для всего мужчин [15:30]

RSS_woman = np.sum((model_woman.fittedvalues - np.array(y).mean())**2)
print(ssr_woman)

121542751.37236509


In [147]:
#    n      e     e1,     e2    p
F(len(df),ssr_all,ssr_man,ssr_woman,2) # F=((e-e1-e2)*(n-2*p-2))/((e1-e2)*(p+1))

0.3255307182085007

In [148]:
#    n      e      e1     e2    p
F2(len(df),ssr_all,ssr_man,ssr_woman,2) # F2=((e-e1-e2)*(n-2*p-2))/((e1+e2)*(p+1))

-0.1460846394411201

In [149]:
# Если значение этой статистики больше критического при данном уровне значимости, 
# то гипотеза об ограничениях отвергается в пользу длинной модели, то есть выборки признаются неоднородными 
# и необходимо строить две разные модели для выборок.

# В нашем случае значение критическое, выборки по половому признаку - неоднородны.